In [7]:
import numpy as np
import sympy
import re
from fractions import Fraction
from tkinter import *


root = Tk()
root.title("Systems of Linear Equations Calculator")
root.geometry("810x500")
root.config(bg='azure2')
root['padx']=10
root['pady']=10



input_values = []
const_values = []
equation_labels = []


#FUNCTIONS
#function to print the grid
def print_grid():
    global rows, cols
    
    rows = int(input_1.get())
    cols = int(input_2.get())

    if 0 < rows and cols <= 5:
        #Creating the label for each variable
        for j in range(cols):
            label = Label(root, text=f"x_{j+1}")
            label.grid(row=1, column=j+1, padx=5, pady=5)
            label.config(bg="azure2")

        #Label for const
        const_label = Label(root, text="b")
        const_label.grid(row=1, column=cols+1)
        const_label.config(bg="azure2")
        matrix_label = Label(root, text="Your Matrix")
        matrix_label.grid(row=1, column=0)
        matrix_label.config(bg="azure2")


        for i in range(rows):
            #Label for each equation
            equation_label = Label(text=i+1)
            equation_label.grid(row=i+5)
            equation_label.config(bg="azure2")
            equation_labels.append(equation_label)

            for j in range(cols):
                values = Entry(root, width=10)
                values.grid(row=i+5, column=j+1, padx=5, pady=5)
                values.config( borderwidth=0)
                input_values.append(values)
            
            # New column for constant entries
            const_entry = Entry(root, width=10)
            const_entry.config(bg="lightblue", borderwidth=0)
            const_entry.grid(row=i+5, column=cols+1, padx=5, pady=5)
            const_values.append(const_entry)

#function to remove the previous matrix
def new_matrix():
    for label in root.grid_slaves(row=1):
        label.grid_forget()

    for values in input_values:
        values.destroy()

    for label in equation_labels:
        label.grid_forget()

    for const_entry in const_values:
        const_entry.grid_forget()

    input_values.clear()
    const_values.clear()
    equation_labels.clear()


# Function to get the value from a specific cell
def get_cell_value(row, col):
    index = row * cols + col
    return input_values[index].get()


# Function to format the input data into useable arrays
def arr2d_func():
    global coefficient_arr, const_arr
    const_arr = []
    coefficient_arr = []

    for i in range(rows):
        row_values = []
        for j in range(cols):
            value = get_cell_value(i, j)
            try:
                value = int(value)
            except ValueError:
                value = 0
            row_values.append(value)
        coefficient_arr.append(row_values)

        const_value = const_values[i].get()
        try:
            const_value = int(const_value)
        except ValueError:
            const_value = 0
        const_arr.append(const_value)

    print("Constants: ",const_arr)
    print("Coefficients:  ",coefficient_arr)
    return const_arr
    return coefficent_arr

#GUI INPUTS
# Taking user input for dimensions
label_row = Label(text="Enter number of equations:", justify="left")
label_row.grid(row=0, column=0)
label_row.config(bg="azure2")
input_1 = Entry(root, width=10)
input_1.grid(row=0, column=1)

label_col = Label(text="Enter number of coefficients:", justify="left")
label_col.grid(row=0, column=2)
label_col.config(bg="azure2")

input_2 = Entry(root, width=10)
input_2.grid(row=0, column=3, padx=2)

#Create matrix button
grid_button = Button(root, text="Create Matrix", command=print_grid, width=15)
grid_button.grid(row=0, column=4)

#New Matrix button
new_button = Button(root, text="Clear all", command=new_matrix, width=15)
new_button.grid(row=0, column=5)

#print array
solve_button = Button(root, text="Solve", command=arr2d_func, width=15)
solve_button.grid(row=0, column=6)

root.mainloop()

>>>Solution Function 5 (Handles Inconsistent/No Solution) **WORKING - IN USE**

In [28]:
def solve_linear_equations(coefficients, constants):
    # Augment the coefficients matrix with the constants vector
    augmented_matrix = np.hstack((coefficients, constants.reshape(-1, 1)))
    # Convert to a sympy Matrix and compute the RREF
    rref_matrix, pivot_columns = sympy.Matrix(augmented_matrix).rref()
    rref_matrix = np.array(rref_matrix.tolist(), dtype=float)

    # Check for inconsistency: a row of zeros in coefficients with a non-zero constant
    for i in range(len(rref_matrix)):
        if np.allclose(rref_matrix[i, :-1], 0) and not np.isclose(rref_matrix[i, -1], 0):
            return None, "No solution"

    num_rows, num_cols = rref_matrix.shape
    free_vars = num_cols - num_rows - 1  # -1 for the constants column

    if free_vars > 0:
        # Infinite solutions, proceed to express in terms of free variables
        solution_set = []
        for i in range(num_rows):
            if i in pivot_columns:
                leading_coeff = rref_matrix[i][i]
                expression = f"{rref_matrix[i, -1]/leading_coeff}"
                for j in range(i + 1, num_cols - 1):
                    if rref_matrix[i, j] != 0:
                        expression += f" - {rref_matrix[i, j]/leading_coeff}*x{j+1}"
                solution_set.append(expression)
        # Add expressions for free variables
        for i in range(num_rows, num_cols - 1):  # -1 to exclude the constants column
            if i not in pivot_columns:
                solution_set.append(f"x{i+1} is a free variable")
        
        parametric_solution = ', '.join(solution_set)
        return parametric_solution, "Infinite solutions"
    elif len(pivot_columns) == num_rows:
        # Unique solution, extract it from the last column of the RREF
        unique_solution = rref_matrix[:, -1]
        return unique_solution, "Unique solution"
    else:
        # Underdetermined system with no unique solution
        return None, "Infinite solutions, cannot express as unique solution"

>Solution Formating 2 (Parametric w/ fractions) **WORKING - IN USE**

In [29]:
def format_parametric_solution(solution_str):
    # Split the solution string into individual variable components
    variables = solution_str.split(',')

    # Define a pattern to match the variables like 'x4' and 'x5'
    var_pattern = re.compile(r'x\d+')

    formatted_solution = []

    for i, var in enumerate(variables):
        # Find all free variables in the expression
        free_vars = var_pattern.findall(var)

        # Correctly identify negative and positive floats
        expr = re.sub(r'(?<!\w)-?\d+\.\d+', lambda m: str(Fraction(float(m.group(0))).limit_denominator()), var)

        # If the variable part is a free variable, format it accordingly
        if i + 1 in [int(free_var[1:]) for free_var in free_vars]:
            formatted_solution.append(f"x{i+1} is a free variable")
        else:
            # Remove unnecessary pluses and correct double negatives
            expr = expr.replace(' - -', ' + ').replace('+-', '-')
            formatted_solution.append(f"x{i+1} = {expr.strip()}")

    return formatted_solution

In [30]:
coefficients_array = np.array(coefficient_arr)
constants_array = np.array(const_arr)

solution, status = solve_linear_equations(coefficients_array, constants_array)
print("Status:", status)
print("Solution:", solution)

# Format the parametric solution
formatted_solution = format_parametric_solution(str(solution))

# Print the formatted solution
for line in formatted_solution:
    print(line)


Status: Infinite solutions, cannot express as unique solution
Solution: None
x1 = None
